In [1]:
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
import re

In [2]:
news_requests = requests.get("https://vancouversun.com/entertainment/local-arts/seven-things-to-do-in-vancouver-june-14-20-anderson-paak-the-free-nationals-boom-more")

In [3]:
news_requests.status_code

200

In [4]:
news_requests.headers

{'Server': 'nginx', 'Date': 'Tue, 03 Sep 2019 14:36:36 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=86400', 'Vary': 'Accept-Encoding, Cookie', 'X-hacker': "If you're reading this, you should visit automattic.com/jobs and apply to join the fun, mention this header.", 'P3P': 'CP="IDC DSP COR ADM DEVi TAIi PSA PSD IVAi IVDi CONi HIS OUR IND CNT"', 'X-Pingback': 'https://vancouversun.com/xmlrpc.php', 'Link': '<https://vancouversun.com/wcm/b3fe2e22-9dff-4472-8037-cc9263b7954f>; rel=shortlink', 'Content-Encoding': 'gzip', 'X-ac': '1.mxp _dca '}

In [5]:
print(news_requests.request.headers)

{'User-Agent': 'python-requests/2.14.2', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [6]:
week_1 = BeautifulSoup(news_requests.text,'lxml')

In [7]:
week_1

<!DOCTYPE html>
<html class="no-js" itemscope="itemscope" itemtype="https://schema.org/NewsArticle" lang="en" prefix="og: https://ogp.me/ns#">
<head data-evt="click" data-evt-typ="signin-reg-module" data-evt-val='{"viewSignRegMod":"Paywall Limit Reached"}' id="janrain-analytics">
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="noarchive" name="robots"/>
<meta content="915AE10930FFCE1EFBB9D7838CE66DE7" name="msvalidate.01"/>
<title>
		Seven things to do in Vancouver June 14-20: Anderson .Paak, BOOM, more | Vancouver Sun	</title>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<link href="https://vancouversun.com/xmlrpc.php" rel="pingback"/>
<script type="text/javascript">
/*! loadCSS. [c]2017 Filament Group, Inc. MIT License */
/* This file is meant as a standalone workflow for
- testing support for link[rel=preload]
- enabling async CSS loading in browsers that do not support rel=preload
- applying rel preload css once loaded,

In [139]:
week_1.find_all("table")

[]

In [140]:
week_1.find_all("h3")

[<h3>Anderson .Paak &amp; The Free Nationals: Best Teef in the Game Tour</h3>,
 <h3>BOOM!</h3>,
 <h3>James O’Mara — Encounters</h3>,
 <h3>5X, A South Asian Millennial Festival</h3>,
 <h3>Breakout 2019</h3>,
 <h3>Judas Priest — Firepower Tour</h3>,
 <h3>Marvel’s Jessica Jones</h3>]

In [35]:
#things.find_all("p")

In [141]:
events = pd.DataFrame({"Event":[]})
rows = []
when_where = []
df = []

for i,entry in enumerate(week_1.find_all("h3")):
    events = events.append({"Event":entry.text.strip()},ignore_index=True) # Stripping <h3> tags
for i,entry in enumerate(week_1.find_all("strong")):
    # Basically saying skip over every third entry (which represented the link to the website which would show up)
    if i%3 < 2:
        # Next sibling of the bold tags, which are the when and where entries
        rows.append(str(entry.next_sibling))
for i in range(len(rows)):
    when_where.append(rows[i])
    if (i+1)%2==0:
        df.append(when_where)
        when_where = []
when_where = pd.DataFrame(df,columns = ["When","Where"])
when_where = when_where.replace(": ","",regex=True)
events.join(when_where)

,Event,When,Where
0,Anderson .Paak & The Free Nationals: Best Teef...,"June 19, 6:30 p.m.","PNE Forum, 2901 E. Hastings St."
1,BOOM!,"Ongoing, daily.","Britannia Mine Museum, Britannia Beach."
2,James O’Mara — Encounters,Until July 13.,"Elan Fine Art, 4 – 258 E. 1st Ave."
3,"5X, A South Asian Millennial Festival",Until June 16.,"Various venues, times."
4,Breakout 2019,"June 15, 2 p.m.","PNE Amphitheatre, 2901 E. Hastings St."
5,Judas Priest — Firepower Tour,"June 17, 7:30 p.m.","Abbotsford Centre, 33800 King Rd."
6,Marvel’s Jessica Jones,June 14,Netflix


In [104]:
when_where = []
df = []

for i in range(len(rows)):
    when_where.append(rows[i])
    if (i+1)%2==0:
        df.append(when_where)
        when_where = []
when_where = pd.DataFrame(df,columns = ["When","Where"])
when_where = when_where.replace(": ","",regex=True)

In [110]:
when_where = when_where.replace(": ","",regex=True)

In [111]:
when_where

,When,Where
0,"June 19, 6:30 p.m.","PNE Forum, 2901 E. Hastings St."
1,"Ongoing, daily.","Britannia Mine Museum, Britannia Beach."
2,Until July 13.,"Elan Fine Art, 4 – 258 E. 1st Ave."
3,Until June 16.,"Various venues, times."
4,"June 15, 2 p.m.","PNE Amphitheatre, 2901 E. Hastings St."
5,"June 17, 7:30 p.m.","Abbotsford Centre, 33800 King Rd."
6,June 14,Netflix


In [142]:
for i,entry in enumerate(week_1.find_all("strong")):
    if i%3 < 2:
        print(str(entry.next_sibling))

: June 19, 6:30 p.m.
: PNE Forum, 2901 E. Hastings St.
: Ongoing, daily.
: Britannia Mine Museum, Britannia Beach.
: Until July 13.
: Elan Fine Art, 4 – 258 E. 1st Ave.
: Until June 16.
: Various venues, times.
: June 15, 2 p.m.
: PNE Amphitheatre, 2901 E. Hastings St.
: June 17, 7:30 p.m.
: Abbotsford Centre, 33800 King Rd.
: June 14
: Netflix


In [60]:
when_where.next_sibling

': June 19, 6:30 p.m.'

In [8]:
def weekly_activities(url):
    week_2 = requests.get(url)
    week = BeautifulSoup(week_2.text,'lxml')
    events = pd.DataFrame({"Event":[]})
    rows = []
    when_where = []
    df = []

    for i,entry in enumerate(week.find_all("h3")):
        events = events.append({"Event":entry.text.strip()},ignore_index=True) # Stripping <h3> tags
    for i,entry in enumerate(week.find_all("strong")):
        # Basically saying skip over every third entry (which represented the link to the website which would show up)
        if i%4 == 1 or i%4 == 2:
            # Next sibling of the bold tags, which are the when and where entries
            rows.append(str(entry.next_sibling))
    for i in range(len(rows)):
        when_where.append(rows[i])
        if (i+1)%2==0:
            df.append(when_where)
            when_where = []
    when_where = pd.DataFrame(df,columns = ["When","Where"])
    when_where = when_where.replace(": ","",regex=True)
    return(events.join(when_where))

## Week: June 7 - 13

In [117]:
week_2 = requests.get("https://vancouversun.com/entertainment/local-arts/seven-things-to-do-in-vancouver-june-7-to-13-world-of-dance-mattiel-more")

In [120]:
week_2.status_code
week_2.reason

'OK'

In [122]:
week = BeautifulSoup(week_2.text,'lxml')

In [136]:
events = pd.DataFrame({"Event":[]})
rows = []
when_where = []
df = []

for i,entry in enumerate(week.find_all("h3")):
    events = events.append({"Event":entry.text.strip()},ignore_index=True) # Stripping <h3> tags
for i,entry in enumerate(week.find_all("strong")):
    # Basically saying skip over every third entry (which represented the link to the website which would show up)
    if i%4 == 1 or i%4 == 2:
        # Next sibling of the bold tags, which are the when and where entries
        rows.append(str(entry.next_sibling))
for i in range(len(rows)):
    when_where.append(rows[i])
    if (i+1)%2==0:
        df.append(when_where)
        when_where = []
when_where = pd.DataFrame(df,columns = ["When","Where"])
when_where = when_where.replace(": ","",regex=True)
events.join(when_where)

,Event,When,Where
0,World of Dance Live Championship Series Vancouver,"June 7, 9 a.m.","Aria Banquet Hall and Convention Centre, 12350..."
1,Mattiel,"June 9, 7 p.m.","WISE Hall, 1882 Adanac"
2,Armistad Maupin’s Tales of the City,"June 7, streaming",Netflix
3,Doomsquad,"June 7, 8 p.m.","Fortune Sound Club, 147 E. Pender"
4,Mom’s the Word 3: Nest 1/2 Empty,"Until July 20, various dates and times","Arts Club Granville Island Stage, 1585 Johnsto..."
5,Nick Murphy a.k.a. Chet Fakr,"June 9, 8 p.m.","the Commodore Ballroom, 868 Granville"
6,Benjamin Lumb: New Life,ongoing,"Grosvenor Ambleside, 1355 Bellevue Ave, W. Van."


In [153]:
# Using function

weekly_activities("https://vancouversun.com/entertainment/local-arts/seven-things-to-do-in-vancouver-june-7-to-13-world-of-dance-mattiel-more")

,Event,When,Where
0,World of Dance Live Championship Series Vancouver,"June 7, 9 a.m.","Aria Banquet Hall and Convention Centre, 12350..."
1,Mattiel,"June 9, 7 p.m.","WISE Hall, 1882 Adanac"
2,Armistad Maupin’s Tales of the City,"June 7, streaming",Netflix
3,Doomsquad,"June 7, 8 p.m.","Fortune Sound Club, 147 E. Pender"
4,Mom’s the Word 3: Nest 1/2 Empty,"Until July 20, various dates and times","Arts Club Granville Island Stage, 1585 Johnsto..."
5,Nick Murphy a.k.a. Chet Fakr,"June 9, 8 p.m.","the Commodore Ballroom, 868 Granville"
6,Benjamin Lumb: New Life,ongoing,"Grosvenor Ambleside, 1355 Bellevue Ave, W. Van."


## Week: May 24 - 30

In [143]:
may_4 = requests.get("https://vancouversun.com/entertainment/local-arts/seven-things-to-do-in-vancouver-may-24-30-alessia-cara-black-mother-and-more")

In [145]:
week_5 = BeautifulSoup(may_4.text,'lxml')

In [147]:
events = pd.DataFrame({"Event":[]})
rows = []
when_where = []
df = []

for i,entry in enumerate(week_5.find_all("h3")):
    events = events.append({"Event":entry.text.strip()},ignore_index=True) # Stripping <h3> tags
for i,entry in enumerate(week_5.find_all("strong")):
    # Basically saying skip over every third entry (which represented the link to the website which would show up)
    if i%3 < 2:
    # Next sibling of the bold tags, which are the when and where entries
        rows.append(str(entry.next_sibling))
for i in range(len(rows)):
    when_where.append(rows[i])
    if (i+1)%2==0:
        df.append(when_where)
        when_where = []
when_where = pd.DataFrame(df,columns = ["When","Where"])
when_where = when_where.replace(": ","",regex=True)
events.join(when_where)

,Event,When,Where
0,Runaway Jane,"May 25, 8 p.m.","Rogers Arena, 800 Griffiths Way"
1,Mat Holstrom: Arcadia,"Until May 26, various times","VanDusen Botanical Garden, 5251 Oak St."
2,Black Mother,"Until May 24, various times","The Cinematheque, 1131 Howe St."
3,Nav: Bad Habits Tour,"May 26, 7 p.m.","PNE Forum, 2901 E. Hastings St."
4,I Walked the Line,"May 25-27, 2 p.m.; May 24, 8 p.m.","Anvil Centre, 777 Columbia St., New Westminster"
5,Sunday Morning,"May 25, 8 p.m.","The Fox Cabaret, 2321 Main St."
6,Hard Rubber Orchestra meets New York Heavies,"May 25, 8 p.m.","Fei and Milton Wong Experimental Theatre, Gold..."


## Week: May 17 - 23

In [148]:
may_3 = requests.get("https://vancouversun.com/entertainment/local-arts/seven-things-to-do-in-vancouver-may-17-23-cloverdale-rodeo-elle-king-and-more")

In [150]:
week_6 = BeautifulSoup(may_3.text,'lxml')

In [151]:
events = pd.DataFrame({"Event":[]})
rows = []
when_where = []
df = []

for i,entry in enumerate(week_6.find_all("h3")):
    events = events.append({"Event":entry.text.strip()},ignore_index=True) # Stripping <h3> tags
for i,entry in enumerate(week_6.find_all("strong")):
    # Basically saying skip over every third entry (which represented the link to the website which would show up)
    if i%3 < 2:
    # Next sibling of the bold tags, which are the when and where entries
        rows.append(str(entry.next_sibling))
for i in range(len(rows)):
    when_where.append(rows[i])
    if (i+1)%2==0:
        df.append(when_where)
        when_where = []
when_where = pd.DataFrame(df,columns = ["When","Where"])
when_where = when_where.replace(": ","",regex=True)
events.join(when_where)

,Event,When,Where
0,Cloverdale Rodeo and Country Fair,"May 17-20, various times.","Cloverdale Exhibition Grounds, Surrey"
1,Matilda: The Musical,"Until July 14, various times.","Stanley Industrial Alliance Stage, 2750 Granvi..."
2,Opera Mariposa: The Impossible Dream,"May 18, 7:30 p.m.","St. Helen’s Anglican Church, 4405 W. 8th Ave."
3,Elle King: Shaking the Spirit Tour,"May 21, 8 p.m.","Commodore Ballroom, 868 Granville St."
4,Jan de Beer: Tides,"Opens May 21, 6 p.m.","Science World, 1455 Quebec St."
5,Suzie Vinnick,"May 17, 7:30 p.m.","The ANZA Club, 3 West 8th Ave."
6,The Specials — Encore 40th Anniversary Tour,"May 22, 8 p.m.","Commodore Ballroom, 868 Granville St."


## Week of Sep 2

In [9]:
weekly_activities("https://vancouversun.com/entertainment/music/seven-things-to-do-in-vancouver-aug-30-sept-5-iron-maiden-the-dark-crystal-age-of-resistance-more")

,Event,When,Where
0,Iron Maiden Legacy of the Beast Tour,Pepsi Live at Rogers Arena,From $45 at
1,The Dark Crystal: Age of Resistance,Netflix,
2,Bryan Ferry,From $45 at,Various days and times
3,Lily Lau Eagle Claw Kung Fu Canada,"Sept. 5, 5:30 p.m.","Malkin Bowl, Stanley Park"
4,Death Cab for Cutie,"BMO Mainstage, Bard on the Beach, Vanier Park",
5,Shakespeare in Love,$25 at,None
6,Lindsay Beaver,NaN,NaN


In [12]:
sep_2 = requests.get("https://vancouversun.com/entertainment/music/seven-things-to-do-in-vancouver-aug-30-sept-5-iron-maiden-the-dark-crystal-age-of-resistance-more")

In [15]:
sep_2

<Response [200]>

In [17]:
week = BeautifulSoup(sep_2.text,'lxml')

In [31]:
week.find_all('strong')

[<strong>When</strong>,
 <strong>Where</strong>,
 <strong>Tickets and info</strong>,
 <strong>The Dark Crystal: Age of Resistance</strong>,
 <strong>When</strong>,
 <strong>Where</strong>,
 <strong>Tickets and info</strong>,
 <strong>When</strong>,
 <strong>Where</strong>,
 <strong>Tickets and info</strong>,
 <strong>When</strong>,
 <strong>Where</strong>,
 <strong>Tickets and info</strong>,
 <strong>When</strong>,
 <strong>Where</strong>,
 <strong>Tickets and info</strong>,
 <strong>When</strong>,
 <strong>Where</strong>,
 <strong>Tickets and info</strong>,
 <strong>When</strong>,
 <strong>Where</strong>,
 <strong>Tickets and info</strong>,
 <strong><a href="http://pubads.g.doubleclick.net/gampad/clk?id=359262301&amp;iu=/3081/ccn.com" target="_blank" title="INDOCHINO">INDOCHINO</a></strong>]

In [34]:
for i,entry in enumerate(week.find_all('strong')):
    if str(entry.text) == "When":
        print(i,str(entry.next_sibling))

0 : Sept. 3, 6 p.m.
4 : August 30
7 : Sept. 5, 6:30 p.m.
10 : Various days and times
13 : Sept. 5, 5:30 p.m.
16 : Until Sept. 18, various times/dates
19 : Aug. 30, 7:30 p.m.


In [ ]:
def weekly_activities(url):
    week_2 = requests.get(url)
    week = BeautifulSoup(week_2.text,'lxml')
    events = pd.DataFrame({"Event":[]})
    rows = []
    when_where = []
    df = []

    for i,entry in enumerate(week.find_all("h3")):
        events = events.append({"Event":entry.text.strip()},ignore_index=True) # Stripping <h3> tags
    for i,entry in enumerate(week.find_all("strong")):
        # Basically saying skip over every third entry (which represented the link to the website which would show up)
        if i%4 == 1 or i%4 == 2:
            # Next sibling of the bold tags, which are the when and where entries
            rows.append(str(entry.next_sibling))
    for i in range(len(rows)):
        when_where.append(rows[i])
        if (i+1)%2==0:
            df.append(when_where)
            when_where = []
    when_where = pd.DataFrame(df,columns = ["When","Where"])
    when_where = when_where.replace(": ","",regex=True)
    return(events.join(when_where))